In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from neo.agentic import Neo, Task, Instruction, ModelConfigs, OtherConfigs

# Instruction

In [ ]:
instruction = Instruction(
    content="""Act like a german shepherd dog.""",
    model_configs=ModelConfigs(model="gpt-4.1"), 
    other_configs=OtherConfigs(timeaware=False)
    )


In [ ]:
task = Task(user_input="Hi", instruction=instruction,)

In [ ]:
neo = Neo(tasks=[task])

In [ ]:
result = await neo.run_all()

In [ ]:
result.display()

# Multi-tasking

In [ ]:
from neo.agentic import ModelConfigs

In [ ]:
background = """The user is saying hi to his agents who role play different characters."""

instruction = Instruction(content=f"{background}. act like a german shepherd", model_configs={"model": "gpt-4o"})
task = Task(user_input="hello", instruction=instruction)


instruction2 = Instruction(content=f"{background}. act like geeky robot.", model_configs={"model": "claude-3-5-haiku-latest"})
task2 = Task(user_input="hello", instruction=instruction2)


task3 = Task(user_input="hello", instruction=f"{background}. act like a cat", model_configs={"model": "grok-2"})

task4 = Task(user_input="summarize this conversation", name="summarize_task") # name is optional but it makes logging easier to read

default_model_configs = ModelConfigs(model="gemini-2.5-flash-preview-04-17") # default model for any task without a specific model

In [ ]:
# sequence
task.add_subsequent_task(task2)
task2.add_subsequent_task(task3)
task3.add_subsequent_task(task4)

In [ ]:
neo = Neo(tasks=[task], default_model_configs=default_model_configs)

In [ ]:
neo.display_dependencies() 

In [ ]:
thread = await neo.run_all(start_fresh=False)

In [ ]:
thread.display()

In [ ]:
len(thread)

# Debugging

In [ ]:
task.base_thread_snapshot.display()

In [ ]:
task.deliverable.display()

In [ ]:
task2.base_thread_snapshot.display()

In [ ]:
task2.deliverable.display()

In [ ]:
task3.base_thread_snapshot.display()

In [ ]:
task3.deliverable.display()

In [ ]:
task4.base_thread_snapshot.display()

In [ ]:
task4.deliverable.display()

# ModelRegistry
ModelRegistry makes it easier to manage modalities and provide user-friendly errors when a modality is not supported. 

In [ ]:
from neo.agentic import Neo, Task, Instruction, ModelConfigs, OtherConfigs, ModelRegistry

In [ ]:
from neo.types import contents as C
from neo.contexts import Context, Thread

In [ ]:
txt_url = "https://assets.anthropic.com/m/1cd9d098ac3e6467/original/Claude-3-Model-Card-October-Addendum.pdf"

context = Context(
    contents = [
        C.DocumentContent(data=txt_url, mime_type="application/pdf"),
        C.TextContent(data="Summarize the document"),
    ])

In [ ]:
registry = ModelRegistry()

In [ ]:
registry.check_model_input_modalities("gpt-4o")

In [ ]:
registry.check_model_input_modalities("gemini-2.0-flash")

In [ ]:
# task = Task(user_input=None, 
#             instruction=Instruction(model_configs=ModelConfigs(model="gpt-4o")))

task = Task(user_input=None, 
            instruction=Instruction(model_configs=ModelConfigs(model="gemini-2.0-flash")))


In [ ]:
neo = Neo(tasks=[task])

In [ ]:
thread = await neo.run_all(start_fresh=True, base_thread=Thread(contexts=[context]))

In [ ]:
thread.display()